In [1]:
import pandas as pd
from utils.quilt import load_datasets
import plotly
import plotly.graph_objs as go
import cufflinks as cf
from zipfile import ZipFile
from io import BytesIO
from urllib.request import urlopen

plotly.offline.init_notebook_mode(connected=True)
cf.set_config_file(offline=True)

In [2]:

# Calculate burden of disease (BoD) from IHME data
# File contains disease burden estimates for Finland, all ages and both sexes

zipurl = "http://en.opasnet.org/en-opwiki/images/2/2d/IHME_Fin_Risks_by_Cause.zip"
file_name = "IHME-GBD_2017_DATA-8ce9adcf-1.csv"

with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        df = pd.read_csv(zfile.open(file_name))
        
df = df.loc[df.measure_name == "DALYs (Disability-Adjusted Life Years)"].loc[df.metric_name == "Number"]
df = df.loc[:,['cause_name','year','val']]

tmp = df
tmp = tmp.loc[tmp.cause_name.isin(['Cardiovascular diseases','Chronic respiratory diseases'])]
tmp = tmp.groupby('year').sum().reset_index()
tmp['cause_name'] = 'Cardiopulmonary mortality'

df = df.groupby('year').sum().reset_index()
df['cause_name'] = 'Total mortality'

df = pd.concat([tmp, df],axis=0)

# From IHME database
# DALYs (Disability-Adjusted Life Years), Finland, Both sexes, All Ages
# Tracheal, bronchus, and lung cancer, Number, 2008-2017
tmp = pd.DataFrame({'year': [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
             'val': [
42651.71,
43159.42,
43754.29,
43621.28,
43759.33,
44319.12,
43718.99,
43327.07,
44461.26,
45680.82]})
tmp['cause_name'] = "Lung cancer mortality"

df = pd.concat([df, tmp], axis=0)

tmp = df.loc[df.year==2017].drop(columns='year')
#tmp = pd.merge(tmp, pd.DataFrame([2017, 2018, 2019, 2020], columns="year"))
tmp['tmp'] = 1
tmp = tmp.merge(pd.DataFrame({'year' : [2018, 2019, 2020], 'tmp':[1,1,1]})).drop(columns='tmp')

df = pd.concat([df,tmp], axis=0)
print(df)

BoD = df

df.to_csv("BoD.csv")

    year           val                 cause_name
0   1990  4.758732e+05  Cardiopulmonary mortality
1   1991  4.605241e+05  Cardiopulmonary mortality
2   1992  4.536654e+05  Cardiopulmonary mortality
3   1993  4.434454e+05  Cardiopulmonary mortality
4   1994  4.212067e+05  Cardiopulmonary mortality
..   ...           ...                        ...
4   2019  1.605238e+06            Total mortality
5   2020  1.605238e+06            Total mortality
6   2018  4.568082e+04      Lung cancer mortality
7   2019  4.568082e+04      Lung cancer mortality
8   2020  4.568082e+04      Lung cancer mortality

[75 rows x 3 columns]


In [109]:
# Calculate fine particulate matter (PM2.5) emissions (ton/a?)

df = load_datasets('jyrjola/lipasto/emissions_by_municipality')

df = df[df.Municipality == 'Helsinki']
df = df[['Year', 'Vehicle', 'Road', 'PM']]
print(df)

       Year      Vehicle      Road         PM
270    2012        Buses  Highways   0.951182
271    2012        Buses     Urban   6.804615
272    2012         Cars  Highways   7.672931
273    2012         Cars     Urban  15.944340
274    2012       Mopeds       All   3.299448
...     ...          ...       ...        ...
19451  2018  Motorcycles       All   2.538113
19452  2018       Trucks  Highways   2.449805
19453  2018       Trucks     Urban   3.161581
19454  2018         Vans  Highways   5.611752
19455  2018         Vans     Urban   8.401600

[75 rows x 4 columns]


In [110]:
gdf = df.groupby(['Year', 'Road']).sum().reset_index()
df = gdf[gdf.Road == 'Urban'].drop(columns='Road')

print(df)

In [1]:
import json
import urllib.request
import pandas as pd

def opbase_data(ident, drop=['sid','aid','Obs']):
    url = "http://cl1.opasnet.org/opasnet_base_2/index.php?ident=" + ident + "&act=0" # URL of the Opasnet Base table

    obj = urllib.request.urlopen(url).read().decode()
    obj = json.loads(obj)

    url2 = "http://cl1.opasnet.org/opasnet_base_2/index.php?key=" + obj['key'] # URL for obtaining the data with a temporary key

    df = urllib.request.urlopen(url2).read().decode()
    df = json.loads(json.loads(df)['data'])
    df = pd.json_normalize(df)

    #from pandas.io.json import json_normalize

    indices = pd.json_normalize(obj['indices'])
    
    for i in indices.ident:
        df.rename({i: indices.name[int(i)]}, axis=1, inplace=True)
        
    df = df.drop(columns=drop)

    return(df)


In [78]:
# Fetch knowledge crystal for intake fraction

ident = "Op_en5813.intake_fractions_of_pm"

iF = opbase_data(ident)
iF = iF[iF.Pollutant == 'PM2.5']
iF.res = list(map(float, iF.res))
iF.res = iF.res * 1E-6


In [79]:
# iF = c * BR * Pop / Em, where iF: intake fraction, c: ambient concentration, Pop: population size
# c = iF * Em / BR / Pop

# Therefore, exposure concentration is calculated as:
def exposure(emissions, population, iF):
    BR = 20 # breathing rate is the nominal 20 m³/d
    BR = BR * 365.25 # m³/d --> m³/a

    emissions = emissions * 1E+12 # ton --> ug

    out = (emissions * out) * iF / BR / population # the actual equation
    out = unkeep(out, prevresults = TRUE, sources = TRUE)
    out@output = out@output[!out@output$Pollutant %in% c("CO2", "CO2official") , ]
    colnames(out@output)[colnames(out@output) == "Pollutant"] <- "Exposure_agent"
    out = oapply(out, cols = c("Renovation"), FUN = sum)

    return(out)


   Pollutant Emission height     Area           res
16     PM2.5            High    Urban  1.100000e-05
17     PM2.5            High    Rural  1.600000e-06
18     PM2.5            High   Remote  1.000000e-07
19     PM2.5            High  Average  6.800000e-06
20     PM2.5             Low    Urban  1.500000e-05
21     PM2.5             Low    Rural  2.000000e-06
22     PM2.5             Low   Remote  1.000000e-07
23     PM2.5             Low  Average  6.800000e-06
24     PM2.5          Ground    Urban  4.400000e-05
25     PM2.5          Ground    Rural  3.800000e-06
26     PM2.5          Ground   Remote  1.000000e-07
27     PM2.5          Ground  Average  2.500000e-05
28     PM2.5         Average    Urban  2.600000e-05
29     PM2.5         Average    Rural  2.600000e-06
30     PM2.5         Average   Remote  1.000000e-07
31     PM2.5         Average  Average  1.500000e-05
